In [2]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F

In [3]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/02 14:34:12 WARN Utils: Your hostname, LAPTOP-RELH58H1 resolves to a loopback address: 127.0.1.1; using 172.19.22.4 instead (on interface eth0)
23/08/02 14:34:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/02 14:34:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Converting Crash Data to parquet file

In [4]:
# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/NYC_Vehicle_Collisions_Crashes.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/NYC_Vehicle_Collisions_Crashes.parquet"
df.write.parquet(parquet_output_path)


23/08/02 14:35:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


AnalysisException: [PATH_ALREADY_EXISTS] Path file:/mnt/c/Users/61435/Desktop/Applied Data/Project1/mast30034-project-1-HFISHER00/data/raw/NYC_Vehicle_Collisions_Crashes.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

REMOVING CRASHES WITH NULL FOR BOROUGH (LOCATION)

In [5]:
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("Remove Null Rows").getOrCreate()

# Read the parquet file into a DataFrame
Crash_results = spark.read.parquet('../data/raw/NYC_Vehicle_Collisions_Crashes.parquet')

# Filter out rows with null values in the "BOROUGH" column
Crash_results_filtered = Crash_results.na.drop(subset=["BOROUGH"])

# Show the filtered DataFrame
Crash_results_filtered.show()

# Save the filtered DataFrame to a new Parquet file
Crash_results_filtered.write.parquet('../data/landing/Crash_results_filtered.parquet', mode='overwrite')

23/08/02 14:38:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------+----------+-------------+--------+---------+----------+--------------------+--------------------+------------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|      BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME| CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBU

Convert Crime data csv's to parquets

In [6]:
# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/shooting_data.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/shooting_data.parquet"
df.write.parquet(parquet_output_path)

In [10]:
from pyspark.sql import SparkSession

# Assuming you already have a SparkSession called "spark"
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")

# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/complaint_data.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/complaint_data.parquet"
df.write.parquet(parquet_output_path)

In [12]:
# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/arrest_data.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/arrest_data.parquet"
df.write.parquet(parquet_output_path)

Removing unneccesary columns from crime data

Shooting Data

In [14]:
# Step 1: Read the existing Parquet file into a DataFrame
parquet_input_path = "../data/raw/shooting_data.parquet"
df = spark.read.parquet(parquet_input_path)

# Step 2: Select the columns you want to keep
selected_columns = ["INCIDENT_KEY", "OCCUR_DATE", "OCCUR_TIME", "BORO", "PRECINCT", "Latitude", "Longitude"]
df_selected = df.select(*selected_columns)

# Step 3: Write the selected DataFrame to a new Parquet file
parquet_output_path = "../data/raw/shooting_data_reduced.parquet"
df_selected.write.parquet(parquet_output_path)

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/mnt/c/Users/61435/Desktop/Applied Data/Project1/mast30034-project-1-HFISHER00/data/raw/shooting_data_reduced.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

In [15]:
df_selected.show()

+------------+-------------------+-------------------+---------+--------+------------------+------------------+
|INCIDENT_KEY|         OCCUR_DATE|         OCCUR_TIME|     BORO|PRECINCT|          Latitude|         Longitude|
+------------+-------------------+-------------------+---------+--------+------------------+------------------+
|   228798151|2021-05-27 00:00:00|2023-08-02 21:30:00|   QUEENS|     105|40.662964620000025|-73.73083868899994|
|   137471050|2014-06-27 00:00:00|2023-08-02 17:40:00|    BRONX|      40| 40.81035186300005|-73.92494232599995|
|   147998800|2015-11-21 00:00:00|2023-08-02 03:56:00|   QUEENS|     108| 40.74260663300004|-73.91549174199997|
|   146837977|2015-10-09 00:00:00|2023-08-02 18:30:00|    BRONX|      44| 40.83778200300003|-73.91945661499994|
|    58921844|2009-02-19 00:00:00|2023-08-02 22:58:00|    BRONX|      47| 40.88623791800006|-73.85290950899997|
|   219559682|2020-10-21 00:00:00|2023-08-02 21:36:00| BROOKLYN|      81|40.678456718000064|-73.92795224